In [1]:
%%writefile src/MathMultiply7.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Multiply7 {
    event PrintLog(string s, address _from, uint _amount);
    event PrintLog(address _addr1, address _addr2);
    event PrintLog(uint);
    receive() external payable {
        emit PrintLog(msg.sender, tx.origin); // tx.origin은 트랜잭션을 최초로 생성한 외부 계정, msg.sender는 현재 msg를 전달하는 측의 주소를 뜻한다.
    }
    fallback() external payable {
        emit PrintLog(msg.sender, tx.origin);
    }
    function multiply(uint input) pure public returns(uint) {
        return input * 77;
    }
    function getAddress() view public returns(address) {
        return address(this);
    }
}

contract Math {
    Multiply7 m7 = new Multiply7();
    function deposit(uint amount) payable public {
        require(msg.value == amount);
    }
    function setM7(address payable _addr) public {
        m7 = Multiply7(_addr);
    }
    function multiply() view public returns(uint) {
        uint res = m7.multiply(8);
        return res;
    }
    function send123M7() public payable {
        payable(address(m7)).transfer(123 wei);
    }
    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;
    }
    function getBalanceOfM7() public view returns(uint) {
        return address(m7).balance;
    }
    function getAddressOfM7() view public returns(address) {
        return address(m7);
   }
}

Overwriting src/MathMultiply7.sol


In [2]:
!solc-windows.exe --optimize --combined-json abi,bin src/MathMultiply7.sol > src/MathMultiply7.json

In [3]:
%%writefile src/MathMultiply7DeployFromFile.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

var _abiBinJson = require('./MathMultiply7.json');
contractNames = Object.keys(_abiBinJson.contracts);
contractName = contractNames[0];
console.log("- contract name: ", contractName);
_abi = _abiBinJson.contracts[contractName].abi;
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas:364124}, function (err, transactionHash) {
            if (!err) console.log("hash: " + transactionHash);
        })
    console.log("---> The contract deployed to: " + deployed.options.address);
}
deploy()

Overwriting src/MathMultiply7DeployFromFile.js


In [4]:
!node src/MathMultiply7DeployFromFile.js

- contract name:  src/MathMultiply7.sol:Math
Deploying the contract from 0xd9f4dF6782f64D745f242D8B1768bEb54a3Ee310
hash: 0xd1058525720d1762f835fc2161a21ce01ef0a0105e061c7113c4f47332ec0a42
---> The contract deployed to: 0x90CdE294Deec56f37409Ebaf028F55b8136FCC6e


In [5]:
%%writefile src/MathMultiply7UseFromFile.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider('ws://localhost:8345'));

var _abiBinJson = require('./MathMultiply7.json');
contractNames = Object.keys(_abiBinJson.contracts);
contractName = contractNames[0];
abi = _abiBinJson.contracts[contractName].abi;
addr = "0x90CdE294Deec56f37409Ebaf028F55b8136FCC6e";

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    var _instance = new web3.eth.Contract(abi, addr);

    await _instance.methods.multiply().call().then(console.log); // 8 x 77 = 616 출력
    await _instance.methods.deposit(123).send({from: accounts[0], value: 123}); // 123 wei 입금
    await _instance.methods.getBalanceOfM7().call().then(console.log); // M7 잔고 (0) 출력
    await _instance.methods.send123M7().send({from: accounts[0]}); // M7 계좌로 123 wei 입금, tx.origin은 accounts[0], msg.sender는 Math 컨트랙의 주소
    await _instance.methods.getBalanceOfM7().call().then(console.log); // M7 잔고 (123) 출력

    await _instance.methods.getAddressOfM7().call().then(console.log); // Math 컨트랙트가 참조하는 Multiply7 컨트랙 주소를 의미

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);
}
doIt()

Overwriting src/MathMultiply7UseFromFile.js


In [6]:
!node src/MathMultiply7UseFromFile.js

Account: 0xd9f4dF6782f64D745f242D8B1768bEb54a3Ee310
Balance before: 999988549652801077205
616
0
123
0x8DAdA62deD3aa849CecA059e668f7ed2f08C9602
Balance after: 999988410746216765442
Balance diff: 138906584350720
